Template list file

In [1]:
from astropy.table import Table, hstack
from pathlib import Path
import numpy as np
from eazy import filters

WD = Path.cwd()
TEMPDIR = WD.parent / "BROWN_COSMOS"
INPUTDIR = WD / "data" / "input"

INPUTDIR.mkdir(exist_ok=True)

temppaths_brown = list(TEMPDIR.glob("*.dat"))
temppaths_cosmos = list(TEMPDIR.glob("*.sed"))
temppaths = temppaths_brown + temppaths_cosmos

In [25]:
# Brown+COSMOS template
with open("BROWN_COSMOS.template", "w") as f:
    for i, temppath in enumerate(temppaths):
        f.write(f"{i+1:3d} {temppath}\n")

Translate file

In [26]:
# make a dummy translate file, and make the column names of input catalogs correspond to the filter numbers
with open("dummy.translate", "w") as f:
    f.write("dummy F999")

Input catalogs

In [27]:
# first read filter numbers that correspond to 7DT filters
filt_latest_spherex7dt = filters.FilterFile(WD /"data" / "filters" / "FILTER.RES+SPHEREx+7DT_reduced.res")
ids_spherex = filt_latest_spherex7dt.search("SPHEREx", verbose=False)
ids_7dt = filt_latest_spherex7dt.search("7DT", verbose=False)
nums_spherex = ids_spherex + 1
nums_7dt = ids_7dt + 1

flux_colnames_spherex = np.array([f"F{num}" for num in nums_spherex])
erro_colnames_spherex = np.array([f"E{num}" for num in nums_spherex])

flux_colnames_7dt = np.array([f"F{num}" for num in nums_7dt])
erro_colnames_7dt = np.array([f"E{num}" for num in nums_7dt])

In [28]:
filt_latest_spherex7dt.filters[ids_7dt[0]].pivot

4012.841032018721

In [29]:
ids_7dt

array([520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532,
       533, 534, 535, 536, 537, 538, 539])

In [30]:
nums_7dt

array([521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533,
       534, 535, 536, 537, 538, 539, 540])

In [31]:
np.array(filt_latest_spherex7dt.filters)[ids_spherex]

array(['SPHEREx_Band1_1 lambda_c= 7.3947e+03 AB-Vega= 0.350 w95=206.8',
       'SPHEREx_Band1_2 lambda_c= 7.5779e+03 AB-Vega= 0.380 w95=212.9',
       'SPHEREx_Band1_3 lambda_c= 7.7657e+03 AB-Vega= 0.412 w95=219.3',
       'SPHEREx_Band1_4 lambda_c= 7.9582e+03 AB-Vega= 0.442 w95=225.8',
       'SPHEREx_Band1_5 lambda_c= 8.1554e+03 AB-Vega= 0.472 w95=231.8',
       'SPHEREx_Band1_6 lambda_c= 8.3575e+03 AB-Vega= 0.501 w95=238.7',
       'SPHEREx_Band1_7 lambda_c= 8.5647e+03 AB-Vega= 0.548 w95=245.4',
       'SPHEREx_Band1_8 lambda_c= 8.7771e+03 AB-Vega= 0.528 w95=252.9',
       'SPHEREx_Band1_9 lambda_c= 8.9949e+03 AB-Vega= 0.496 w95=259.3',
       'SPHEREx_Band1_10 lambda_c= 9.2175e+03 AB-Vega= 0.510 w95=267.1',
       'SPHEREx_Band1_11 lambda_c= 9.4460e+03 AB-Vega= 0.515 w95=274.4',
       'SPHEREx_Band1_12 lambda_c= 9.6804e+03 AB-Vega= 0.533 w95=282.7',
       'SPHEREx_Band1_13 lambda_c= 9.9203e+03 AB-Vega= 0.570 w95=290.3',
       'SPHEREx_Band1_14 lambda_c= 1.0166e+04 AB-Vega= 0.622

7DS GAMA catalog

In [89]:
datatab = Table.read(WD / "data" / "catalogs" / "GAMA_7DS_mock_catalog.ecsv")

base = datatab["source_id", "imag", "redshift"]
base.rename_column("source_id", "ID")
# base.rename_column("imag", "HSC_i_MAG")
base.rename_column("redshift", "z_spec")

In [90]:
fluxlabels = [
    "fiducial_flux_RIS",
    "fiducial_flux_WFS",
    "fiducial_flux_IMS",
]
errolabels = [
    "fiducial_error_RIS",
    "fiducial_error_WFS",
    "fiducial_error_IMS",
]
surveylabels = ["RIS", "WFS", "IMS"]

for fl, el, sl in zip(fluxlabels, errolabels, surveylabels):
    fiducial_flux_array = datatab[fl]
    fiducial_erro_array = datatab[el]
    fluxtab = Table(data=fiducial_flux_array, names=flux_colnames_7dt)
    errotab = Table(data=fiducial_erro_array, names=erro_colnames_7dt)
    inputtab = hstack([base, fluxtab, errotab])
    inputtab.write(INPUTDIR / f"GAMA_input_7DS_{sl}.fits", overwrite=True)

In [91]:
inputtab

ID,imag,z_spec,F521,F522,F523,F524,F525,F526,F527,F528,F529,F530,F531,F532,F533,F534,F535,F536,F537,F538,F539,F540,E521,E522,E523,E524,E525,E526,E527,E528,E529,E530,E531,E532,E533,E534,E535,E536,E537,E538,E539,E540
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
128980003711188,17.2534,0.0818,66.12056446908535,86.46318986442277,147.68155040807838,196.87100474950455,273.90385265819145,310.8089068942095,313.88636467210705,364.2671199569958,403.7960840676696,446.51782666165604,455.74624313331145,483.4720249659496,536.6327700937638,565.6227268322173,582.9906197477782,617.7467988325893,711.6537850861304,751.3940926801062,751.4695167266356,805.6377905396944,0.6729875226000585,0.8713537276230338,1.4946444380555355,1.967693223215812,2.737587824087003,3.090401728500584,3.0919866906944007,3.6853994469486953,4.064215855172738,4.450646972772354,4.643784070579114,4.843293476234208,5.38071803967358,5.6771738213785765,6.022601949217367,6.221943773491877,7.151529813827888,7.561525692257205,7.627366212359283,8.247893004047016
128980005210530,18.26186,0.2899,18.25993835321805,26.455073008130064,31.17504873899463,40.58402678383235,47.46382543782245,81.88291565340398,100.61860376367748,131.97481639768722,157.1548957299888,302.0236406772888,182.73328327486746,197.79496854275757,232.5572279004618,244.97561950945973,280.7633063794831,292.895757676655,289.59159295416396,315.61167055523345,334.4963148277337,338.6848302805986,0.22625414550018097,0.28848439708503987,0.3328904563689764,0.4274684949216226,0.476537936990166,0.8316381211437162,1.0156315470539883,1.309180355357444,1.5824339040155475,2.987365591994637,1.8349156593574862,1.9622482243649104,2.3541907438202574,2.4952344847162835,2.8058081062456304,2.933638090149002,2.956210228007917,3.1849331099310367,3.4803487333868515,3.610946690825124
128980017205121,17.78686,0.0756,39.50939334869652,52.87176048489316,77.8896273864573,102.73589288059775,140.8184958875356,157.39021477834763,155.6427250840178,185.86954411786382,215.96622588963484,239.90663540580516,249.64260533403194,264.7150438698187,287.70107596608904,294.0768055755645,308.45357684846186,308.73898882376653,359.3859321075634,363.7938885304316,377.22337616988443,419.3523485777427,0.41649556366857926,0.5489292194706615,0.7961855474005184,1.0476001522668272,1.3877219865910777,1.5624513307083252,1.5633421827357528,1.8951875085250591,2.138302232305945,2.3823801302777428,2.5742085710451894,2.6241987469841415,2.8941932393339407,2.9477642686890064,3.1416996190148407,3.138606527882131,3.6717829585215616,3.76165097069504,3.818519523250883,4.319258967271096
128980048809144,18.124,0.2695,58.62708694215769,66.07706339857197,74.10252067794183,98.87018280809617,128.4758104572139,152.39310879853008,157.28787472237408,180.8010691685865,199.5678523613393,213.31464516208067,213.05120708857874,231.42695828738448,248.05143781588717,261.43263984893156,280.14981926443556,283.63121594245985,291.4597870107688,367.6968601324386,332.6001624711873,334.94556514379656,0.6047803472043324,0.6805980310806248,0.7424092985997652,0.9986683437689678,1.2945907663216096,1.5275339144461832,1.609335643468981,1.8422868450356014,1.9968877913484508,2.1734946680042926,2.1604267565625195,2.3226470643369566,2.495861534352382,2.6596399596849305,2.8001316381276826,2.876253928049943,3.001276379586928,3.7584445466212975,3.421806701078587,3.4695447854794588
128980053210356,18.25551,0.0826,28.44595325469674,35.21150411036677,56.10602076635788,70.95697225964032,96.71901222042578,110.58535153014995,112.09832304746125,131.72576607471092,152.134616260094,166.4114427950912,181.82771265221643,186.14992960887164,206.22165791170485,209.87475880184914,222.20684097311013,222.25445944684617,248.46705202052942,267.9928

In [36]:
np.arange(len(inputtab["F521"]))[np.isnan(inputtab["F521"].data)]

array([ 2779,  6671,  9676, 10426, 10958, 11062, 11352, 11716, 12654,
       13172, 13185, 13876, 13886, 14481, 17810, 17820, 17827, 22882,
       25450, 26121, 27375, 28044, 31791, 32860, 34400, 34560, 34707,
       34913, 35264, 36146, 36853, 37595, 40618, 40649, 40714, 40728,
       41749, 43780, 43791, 44030, 44066])

7DS COSMOS catalog

In [87]:
datatab = Table.read(WD / "data" / "catalogs" / "Fiducial_fluxes_7DS_v2_more.fits")

cosmos2020 = Table.read(WD / "data" / "catalogs" / "cosmos2020.fits")
cosmos2020.sort("Farmer")

cospd = cosmos2020.to_pandas()
datpd = datatab["source_id", "HSC_i_MAG", "lp_zBEST", "FLAG_ML"].to_pandas()
df = datpd.merge(
    cospd.rename(columns={"Farmer": "source_id", "RAJ2000": "ra", "DEJ2000": "dec"})
).rename(columns={"source_id": "ID", "lp_zBEST": "z_spec"})

base = Table.from_pandas(df)

fluxlabels = [
    "fiducial_flux_RIS",
    "fiducial_flux_WFS",
    "fiducial_flux_IMS",
]
errolabels = [
    "fiducial_error_RIS",
    "fiducial_error_WFS",
    "fiducial_error_IMS",
]
surveylabels = ["RIS", "WFS", "IMS"]

np.random.seed(185)
selectionid = np.random.choice(len(base), 10000, replace=False)
for fl, el, sl in zip(fluxlabels, errolabels, surveylabels):
    fiducial_flux_array = np.array(
        [np.fromstring(row[1:-1], dtype=np.float64, sep=",") for row in datatab[fl]]
    )
    fiducial_erro_array = np.array(
        [np.fromstring(row[1:-1], dtype=np.float64, sep=",") for row in datatab[el]]
    )
    fluxtab = Table(data=fiducial_flux_array, names=flux_colnames_7dt)
    errotab = Table(data=fiducial_erro_array, names=erro_colnames_7dt)
    inputtab = hstack([base, fluxtab, errotab])
    inputtab.write(INPUTDIR / f"COSMOS_input_7DS_{sl}.fits", overwrite=True)

In [88]:
inputtab

ID,HSC_i_MAG,z_spec,FLAG_ML,ra,dec,E_B-V_,Classic,EZzphot,F521,F522,F523,F524,F525,F526,F527,F528,F529,F530,F531,F532,F533,F534,F535,F536,F537,F538,F539,F540,E521,E522,E523,E524,E525,E526,E527,E528,E529,E530,E531,E532,E533,E534,E535,E536,E537,E538,E539,E540
int64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
12,23.990103981421147,0.707,0.0,150.47679031,2.33150699,0.014723000000000002,962809,0.6809810000000001,0.40049770149460284,0.5078999488291236,0.2963054331463698,0.3335773725294298,0.09632781075085359,0.21192752586334654,0.37209524197826066,0.29368319984589747,0.4985870900193809,0.4046981079202723,0.3725967722012696,0.6751451221323529,0.7335593340776831,1.0759871857767476,0.6807956725650958,1.184172133594535,0.9488228953306005,0.5163122176461388,1.267783441864066,1.0484799666863904,0.12772560052432094,0.11387962045581174,0.10839052197376808,0.10817636615811946,0.10437727725409991,0.1112288824337841,0.13267015359507206,0.13184519400400985,0.1461283265740689,0.17801847567707968,0.16864207043732382,0.1791378725006872,0.2072308907717925,0.2797763431091668,0.3134661478735132,0.4545645852922471,0.5045852615320934,0.5662332409277782,0.7237828478570381,1.0045318576971731
27,24.875468381594267,0.7751,0.0,150.71248825,1.79210579,0.018367,387749,0.7748600000000001,0.1555804207561737,0.23298046022847965,0.20890100061051156,0.2425297540900066,0.13264911732809592,-0.0433434970476219,0.35263735305899857,0.16517200662142031,0.3326562541108519,0.3481771377052659,0.34860829092313544,0.02851365349579364,0.008300258547255055,0.42246510939716175,0.46133413411519236,0.14976108017397355,0.851427575477548,-0.897450868898974,0.2007836151573084,2.190287917569994,0.12762509466835778,0.11378978870803297,0.10830610016743172,0.1081038162090855,0.10429418725856796,0.11114809639838144,0.1326006719991642,0.13176752195440508,0.14605500324842657,0.17786784737357897,0.16846603974830524,0.1789315807075524,0.20697474783076347,0.2795536598475472,0.3132480706679894,0.45435525020422407,0.5043646250919653,0.5659866828358857,0.7234516116407833,1.0043847645427422
57,24.701907916699273,0.9672,0.0,150.71818794,1.79272545,0.018322,389282,1.1766590000000001,-0.054737980225801494,0.04076737416359779,0.19915366253184433,0.03687395897230117,-0.09475717700582041,0.22444761604077712,0.30358720097369435,-0.06753550541133471,0.12421776007421409,0.17713751348287443,0.08627032336250276,0.2326219235041975,0.21709594958645737,0.07597485583211182,0.677247753679458,0.03490887039540123,-0.2480045975157707,0.7788508763142791,1.1624588195869474,1.2659804772021994,0.1275851634339583,0.11375130576334735,0.10826689082059997,0.1080594647816753,0.1042494542311209,0.11110165214751623,0.13255633589986265,0.13172823514975307,0.14601737853200192,0.17783937574050313,0.16842310562925578,0.17887109400639423,0.20692856795829775,0.2795355640202538,0.3132266091940865,0.45435071828231227,0.504491708276954,0.566135925331746,0.7235979580280588,1.0046030766284597
73,22.48269003598378,0.429,1.0,149.46802488,1.6256133,0.016032,223727,0.43394200000000005,0.7226129138786842,1.014288922428972,1.1581251416199758,1.3547618005051005,1.400602041888884,1.616632135249312,2.2694297446494174,2.474247850057344,2.9859002690302696,2.6522513206686162,3.3694931983719636,3.5530225794887795,3.283500613480419,3.7532825007462294,3.793716830437237,3.65305438092117,3.914739544396484,4.734975203242984,5.164469807302925,4.4082057736409705,0.12824829191612544,0.11452386712447703,0.10910970207477683,0.10900060223368876,0.10529780578041016,0.11253702196946067,0.1347654223654789,0.1348061563993731,0.14945706282396798,0.18067431455374316,0.17213943650547076,0.18271779798586546,0.21057968912922717,0.2826978

SPHEREx catalog - Reprocessing is needed since the catalog is now based on the QuickCatalog photometry

In [48]:
# all-sky survey - COSMOS sample
datatab = Table.read(WD / "data" / "catalogs" / "COSMOS2020_Richard_2023DEC4_averaged_photometry102_PHOTOZWG.fits")

base = datatab["ID", "ra", "dec", "HSC_i_MAG", "zBEST", "FLAG_ML"]
base.rename_column("zBEST", "z_spec")

In [64]:
fiducial_flux_array = np.vstack(datatab["fiducial_flux"])
fiducial_erro_array = np.vstack(datatab["fiducial_error"])
fluxtable = Table(data=fiducial_flux_array, names=flux_colnames_spherex)
errotable = Table(data=fiducial_erro_array, names=erro_colnames_spherex)
inputtab = hstack([base, fluxtable, errotable])
inputtab.write(INPUTDIR / "COSMOS_input_SPHEREx.fits", overwrite=True)
inputtab[:5]

ID,ra,dec,HSC_i_MAG,z_spec,FLAG_ML,F419,F420,F421,F422,F423,F424,F425,F426,F427,F428,F429,F430,F431,F432,F433,F434,F435,F436,F437,F438,F439,F440,F441,F442,F443,F444,F445,F446,F447,F448,F449,F450,F451,F452,F453,F454,F455,F456,F457,F458,F459,F460,F461,F462,F463,F464,F465,F466,F467,F468,F469,F470,F471,F472,F473,F474,F475,F476,F477,F478,F479,F480,F481,F482,F483,F484,F485,F486,F487,F488,F489,F490,F491,F492,F493,F494,F495,F496,F497,F498,F499,F500,F501,F502,F503,F504,F505,F506,F507,F508,F509,F510,F511,F512,F513,F514,F515,F516,F517,F518,F519,F520,E419,E420,E421,E422,E423,E424,E425,E426,E427,E428,E429,E430,E431,E432,E433,E434,E435,E436,E437,E438,E439,E440,E441,E442,E443,E444,E445,E446,E447,E448,E449,E450,E451,E452,E453,E454,E455,E456,E457,E458,E459,E460,E461,E462,E463,E464,E465,E466,E467,E468,E469,E470,E471,E472,E473,E474,E475,E476,E477,E478,E479,E480,E481,E482,E483,E484,E485,E486,E487,E488,E489,E490,E491,E492,E493,E494,E495,E496,E497,E498,E499,E500,E501,E502,E503,E504,E505,E506,E507,E508,E509,E510,E511,E512,E513,E514,E515,E516,E517,E518,E519,E520
,,,mag,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
12,150.47680901653771,2.3315113049014573,23.990103981421147,0.707,0.0,17.68329373801137,15.75411096596105,-15.362085280375002,13.610149777913989,6.507778576424568,-0.1509509308868243,1.1016895248230354,6.850222669134803,-21.99790006697276,-5.145088516510715,15.975770868291631,27.619462571012996,-5.046165369608225,-13.338167750668182,17.68879536006508,0.6667884301240655,-12.048975453746547,-8.027909731133684,-9.057020742843388,19.4695896320234,0.8835055850442124,-15.644102198894464,16.548164402700177,10.85389920129179,-4.968309969975105,-22.220343562902062,3.478382389384758,-13.097645078318415,28.652314568569402,2.821002057579081,-5.947463204486138,9.52751283031179,-3.6911232099610913,-28.126181673054617,-12.910876933387936,-1.2112242021062574,12.606571701285048,44.81168115400228,-3.4188895591932282,6.2456264483026604,9.82837035459819,-17.591038649911734,-18.231239995329997,-2.2851856982078225,2.5732508696820093,6.24339560075017,-8.893820797521718,27.027209575461672,-6.619714205300591,-4.313844794813997,-28.23511720104826,-17.7066852490255

In [75]:
# all-sky survey - GAMA sample
datatab = Table.read(WD / "data" / "catalogs" / "GAMA_Richard2023_averaged_photometry102_PHOTOZWG.fits")

base = datatab["ID", "ra", "dec", "imag", "redshift", "FLAG_ML"]
base.rename_column("redshift", "z_spec")

fiducial_flux_array = np.vstack(datatab["fiducial_flux"])
fiducial_erro_array = np.vstack(datatab["fiducial_error"])
fluxtable = Table(data=fiducial_flux_array, names=flux_colnames_spherex)
errotable = Table(data=fiducial_erro_array, names=erro_colnames_spherex)
inputtab = hstack([base, fluxtable, errotable])
inputtab.write(INPUTDIR / "GAMA_input_SPHEREx.fits", overwrite=True)
inputtab[:5]

ID,ra,dec,imag,z_spec,FLAG_ML,F419,F420,F421,F422,F423,F424,F425,F426,F427,F428,F429,F430,F431,F432,F433,F434,F435,F436,F437,F438,F439,F440,F441,F442,F443,F444,F445,F446,F447,F448,F449,F450,F451,F452,F453,F454,F455,F456,F457,F458,F459,F460,F461,F462,F463,F464,F465,F466,F467,F468,F469,F470,F471,F472,F473,F474,F475,F476,F477,F478,F479,F480,F481,F482,F483,F484,F485,F486,F487,F488,F489,F490,F491,F492,F493,F494,F495,F496,F497,F498,F499,F500,F501,F502,F503,F504,F505,F506,F507,F508,F509,F510,F511,F512,F513,F514,F515,F516,F517,F518,F519,F520,E419,E420,E421,E422,E423,E424,E425,E426,E427,E428,E429,E430,E431,E432,E433,E434,E435,E436,E437,E438,E439,E440,E441,E442,E443,E444,E445,E446,E447,E448,E449,E450,E451,E452,E453,E454,E455,E456,E457,E458,E459,E460,E461,E462,E463,E464,E465,E466,E467,E468,E469,E470,E471,E472,E473,E474,E475,E476,E477,E478,E479,E480,E481,E482,E483,E484,E485,E486,E487,E488,E489,E490,E491,E492,E493,E494,E495,E496,E497,E498,E499,E500,E501,E502,E503,E504,E505,E506,E507,E508,E509,E510,E511,E512,E513,E514,E515,E516,E517,E518,E519,E520
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
128980003711188,129.57993,-1.02978,17.2534,0.0818,0.0,586.8895274638456,610.3927475680308,645.1677525286585,681.8690106391797,745.7418035605931,754.9895297913133,787.6844199776754,847.6723706769066,853.3159647795113,834.487924137454,886.0091557290685,927.2772538774623,937.27529889049,1054.6816464257133,1090.1004972771962,1127.2057789757073,1176.9141523876008,1090.769868539235,1214.8633391170201,1217.6200702984272,1150.4434666129032,1208.9157459469675,1207.6215850781798,1257.8877419539658,1256.2120182060223,1284.9607834339035,1264.7165983579723,1293.159580115019,1321.5430253658233,1331.3234212860411,1334.7223848203787,1378.4541948963133,1447.557644883939,1434.6712035198673,1492.3360844131184,1469.4843001953946,1534.9477747122535,1585.8193482322577,1642.8397347884945,1588.6022372482134,1638.0372198246332,1566.751527566263,1483.7956440437592,1463.304324094411,1398.951623073077,1368.900822569632,1339.631426502362,1316.354199759295,1311.9792166026452,1276.8013701605491,1298.643327463935,1267.035835702404,1174.7603922458882,1015.4563826898984,924.3273256139364,783.4716591890664,764.9132380482761,779.5125465597815,770.3061217778531,759.5746318442667,747.7687211277616,748.0553603186759,706.9600651055997,697.0858938817673,702.3779341498735,666.2865920092403,662

In [69]:
# deep-sky survey
datatab = Table.read(WD / "data" / "catalogs" / "cosmos2020_random1000_in_sep_averaged_photometry102_PHOTOZWG.fits")

base = datatab["ID", "ra", "dec", "HSC_i_MAG", "zBEST"]
base.rename_column("zBEST", "z_spec")

fiducial_flux_array = np.vstack(datatab["fiducial_flux"])
fiducial_erro_array = np.vstack(datatab["fiducial_error"])
fluxtable = Table(data=fiducial_flux_array, names=flux_colnames_spherex)
errotable = Table(data=fiducial_erro_array, names=erro_colnames_spherex)
inputtab = hstack([base, fluxtable, errotable])
inputtab.write(INPUTDIR / "COSMOS_input_SPHEREx_deep.fits", overwrite=True)
inputtab[:5]

ID,ra,dec,HSC_i_MAG,z_spec,F419,F420,F421,F422,F423,F424,F425,F426,F427,F428,F429,F430,F431,F432,F433,F434,F435,F436,F437,F438,F439,F440,F441,F442,F443,F444,F445,F446,F447,F448,F449,F450,F451,F452,F453,F454,F455,F456,F457,F458,F459,F460,F461,F462,F463,F464,F465,F466,F467,F468,F469,F470,F471,F472,F473,F474,F475,F476,F477,F478,F479,F480,F481,F482,F483,F484,F485,F486,F487,F488,F489,F490,F491,F492,F493,F494,F495,F496,F497,F498,F499,F500,F501,F502,F503,F504,F505,F506,F507,F508,F509,F510,F511,F512,F513,F514,F515,F516,F517,F518,F519,F520,E419,E420,E421,E422,E423,E424,E425,E426,E427,E428,E429,E430,E431,E432,E433,E434,E435,E436,E437,E438,E439,E440,E441,E442,E443,E444,E445,E446,E447,E448,E449,E450,E451,E452,E453,E454,E455,E456,E457,E458,E459,E460,E461,E462,E463,E464,E465,E466,E467,E468,E469,E470,E471,E472,E473,E474,E475,E476,E477,E478,E479,E480,E481,E482,E483,E484,E485,E486,E487,E488,E489,E490,E491,E492,E493,E494,E495,E496,E497,E498,E499,E500,E501,E502,E503,E504,E505,E506,E507,E508,E509,E510,E511,E512,E513,E514,E515,E516,E517,E518,E519,E520
bytes6,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
100274,80.75562932023549,-60.116950715106924,24.6785862850226,1.7895,-3.6956113778698194,-2.3455826619719633,-2.166800698774822,-0.4803890426997832,2.1528242310333248,1.2071992725568612,1.4733302913950705,-0.5803135331616126,3.0684561449592467,0.4752909529464923,-0.5054753105815367,-0.1553501404351085,1.7379172553403341,1.3033161819900578,1.0357327959883362,1.4917752409368001,0.5615246516636286,-0.6767726043206083,2.7219770587434073,2.1678971818359156,5.805702754707639,1.7659062289237204,2.094199227117592,4.052785839915584,0.9605837046526041,2.181601905148717,0.727062340911642,2.7492692315501515,-0.9010925070408932,-0.48359709089970454,-0.2297851842970982,2.531073964500025,-0.9667151287135751,0.4550805914798318,3.9102967880510118,0.5079731225792912,2.0606608356777425,1.0780607111676568,1.6905997540289144,0.9228895088987942,0.39881544748127307,0.017090342517101214,1.2435624981576245,-0.49611784405939463,2.158460428483948,0.905733354003133,1.0839641742395048,1.7320292189992434,0.6624984097958209,1.6400529116964413,-0.5524533232828908,-0.2914233505969449,1.2182720348606262,0.32338030275301527,0.09911482989642124,0.5253824842747223,1.5520693748569003,1.137615328090084,1.2645205444204635,1.4871704708957771,2.110804275477605,2.272305781854948,2.1462068159729406,0.0362

Make joint catalogs

In [123]:
cosmos_spherex = Table.read(INPUTDIR / "COSMOS_input_SPHEREx.fits")
gama_spherex = Table.read(INPUTDIR / "GAMA_input_SPHEREx.fits")
cosmos_7ds_wfs = Table.read(INPUTDIR / "COSMOS_input_7DS_WFS.fits")
gama_7ds_wfs = Table.read(INPUTDIR / "GAMA_input_7DS_WFS.fits")
cosmos_7ds_ims = Table.read(INPUTDIR / "COSMOS_input_7DS_IMS.fits")
cosmos_spherex_deep = Table.read(INPUTDIR / "COSMOS_input_SPHEREx_deep.fits")

cosmos_spherex.rename_column("HSC_i_MAG", "imag")
cosmos_7ds_wfs.rename_column("HSC_i_MAG", "imag")
cosmos_7ds_ims.rename_column("HSC_i_MAG", "imag")

In [124]:
from astropy.table import vstack, hstack, join

gama_cosmos_spherex = vstack([gama_spherex, cosmos_spherex])
gama_cosmos_spherex.sort("ID")
gama_cosmos_spherex.write(INPUTDIR / "GAMA_COSMOS_input_SPHEREx.fits", overwrite=True)

gama_cosmos_7ds_wfs = vstack([gama_7ds_wfs, cosmos_7ds_wfs])
gama_cosmos_7ds_wfs.remove_columns(["ra", "dec", "E_B-V_", "Classic", "EZzphot", "z_spec", "FLAG_ML", "imag"])

gama_cosmos_7ds_wfs.sort("ID")
gama_cosmos_7ds_wfs_spherex = join(gama_cosmos_spherex, gama_cosmos_7ds_wfs, keys="ID")

gama_cosmos_7ds_wfs_spherex.write(INPUTDIR / "GAMA_COSMOS_input_7DS_WFS_SPHEREx.fits", overwrite=True)

cosmos_7ds_ims.remove_columns(["ra", "dec", "E_B-V_", "Classic", "EZzphot", "z_spec", "FLAG_ML", "imag"])
cosmos_spherex_deep.sort("ID")
cosmos_7ds_ims.sort("ID")
cosmos_spherex_deep["ID"] = cosmos_spherex_deep["ID"].astype(np.int64)
cosmos_7ds_ims_spherex_deep = join(cosmos_spherex_deep, cosmos_7ds_ims, keys="ID")

cosmos_7ds_ims_spherex_deep.write(INPUTDIR / "COSMOS_input_7DS_IMS_SPHEREx_deep.fits", overwrite=True)

Check the minimum and maximum z values for each catalog

In [14]:
gama_cosmos_7ds_wfs_spherex = Table.read(INPUTDIR / "GAMA_COSMOS_input_7DS_WFS_SPHEREx.fits")
cosmos_7ds_ims_spherex_deep = Table.read(INPUTDIR / "COSMOS_input_7DS_IMS_SPHEREx_deep.fits")
gama_cosmos_spherex = Table.read(INPUTDIR / "GAMA_COSMOS_input_SPHEREx.fits")

In [12]:
z = gama_cosmos_7ds_wfs_spherex["z_spec"]
print(gama_cosmos_7ds_wfs_spherex[z>0]["z_spec"].min())
print(gama_cosmos_7ds_wfs_spherex[z>0]["z_spec"].max())

0.0022
5.7104


In [13]:
z = cosmos_7ds_ims_spherex_deep["z_spec"]
print(cosmos_7ds_ims_spherex_deep[z>0]["z_spec"].min())
print(cosmos_7ds_ims_spherex_deep[z>0]["z_spec"].max())

0.0188
4.3845


In [15]:
z = gama_cosmos_spherex["z_spec"]
print(gama_cosmos_spherex[z>0]["z_spec"].min())
print(gama_cosmos_spherex[z>0]["z_spec"].max())

0.0022
5.7104
